## Zuger Handelsregister

## Imports

In [1]:
import selenium
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import Select
import time
import pandas as pd
from bs4 import BeautifulSoup
from tqdm import tqdm
import bs4 as bs
import requests
from selenium.webdriver.support.ui import WebDriverWait # Um die Seite ganz zu laden
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from selenium.common.exceptions import StaleElementReferenceException
from selenium.common import exceptions 
folderpath=r"C:\Users\Zoe\Desktop\maz-datenjournalismus\Daten_BR_Handelsreg"

## Seiten besuchen und HTML-Code ablegen

## Alles zusammen

In [ ]:
#Wir starten den Browser auf
driver = webdriver.Chrome(executable_path='C:\chromedriver.exe') #bei Windows ist das anders
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
time.sleep(3)
#Hier drücke ich den Cookie-Banner weg
driver.find_element_by_id("j_idt198").click() #ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
#Ich öffne das zusätzliche Panel, ohne dass man dies auf der Seite sieht
su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
driver.execute_script('arguments[0].click();', su_fel_ad)
time.sleep(2)
# Den Filter ansteuern, wo ich Domiziladresse auwählen kann
filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
filt.send_keys('Domiziladresse')
filt.send_keys(Keys.ENTER)
time.sleep(2)
# Hier klappe ich die Zusätzlichen Suchfelder auf
driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
time.sleep(2)
# Nun gege ich die Adresse ein
search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
search_adress.send_keys("Baarerstrasse 11, Zug")
search_adress.send_keys(Keys.ENTER)

In [ ]:
search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
search_adress.send_keys("Baarerstrasse 11, Zug")
search_adress.send_keys(Keys.ENTER)

In [ ]:
# Hier speichere ich die Page
page = driver.page_source.encode("utf-8")
with open ("page.htm", "wb+") as file:
    file.write(page)
    file.close()

In [ ]:
# Yes! Hier kann ich den "Weiter-Button" anwählen
driver.find_element_by_class_name("ui-paginator-next.ui-state-default.ui-corner-all").click()

In [ ]:
#Ich muss den alten Eintrag löschen, bevor ich den Neuen machen kann.
search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
search_adress.clear()

In [ ]:
search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
search_adress.send_keys("Baarerstrasse 79, Zug")
search_adress.send_keys(Keys.ENTER)

In [ ]:
#Am Ende muss ich wieder zurück zum Anfang. Sonst bleibt er auf der letzten Seite.
#Hier tippe ich den Pfeil zurück
driver.find_element_by_class_name("ui-paginator-first.ui-state-default.ui-corner-all").click()

# Nun als LOOP mit Testadressen

In [ ]:
dff_test=pd.read_csv(folderpath+"\BRFrames/test_gross_adresse")

In [ ]:
dff_test

In [ ]:
#Wir starten den Browser auf
driver = webdriver.Chrome(executable_path='C:\chromedriver.exe') #bei Windows ist das anders
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
time.sleep(3)
#Hier drücke ich den Cookie-Banner weg
driver.find_element_by_id("j_idt198").click() #ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
#Ich öffne das zusätzliche Panel, ohne dass man dies auf der Seite sieht
su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
driver.execute_script('arguments[0].click();', su_fel_ad)
time.sleep(2)
# Den Filter ansteuern, wo ich Domiziladresse auwählen kann
filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
filt.send_keys('Domiziladresse')
filt.send_keys(Keys.ENTER)
time.sleep(2)
# Hier klappe ich die Zusätzlichen Suchfelder auf
driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
time.sleep(2)

In [ ]:
df1=pd.DataFrame(listeadress)

In [ ]:
df1

In [ ]:
# Problem: hat eine Adresse keine Firmen ansässig, erscheinen keine Pfeile. 
# Ich könnte es vereinfachen und nur mal runtersaugen, welche Firmen haben und welche nicht. 
# Dies indem ich nur die erste Seite abspeichere. 
# Danach mit all jenen Adressen arbeiten, die einen Eintrag haben. Dort sind die Pfeile vorhanden.
# OOOOOODEERR Code weiter verfeinern:-)
# Vielleicht hilft eine Funktion! 

# in zwei Schritten
hier nun zuerst alle erste Seiten - so dass ich überall eine Anzahl habe. 
Danach gehe ich nur noch jene durch, die Firmen auf den Seiten haben.

In [ ]:
df11=pd.read_csv(folderpath+"\BRFrames/adresse_Zug")

In [ ]:
df11

In [ ]:
df11_n=df11.tail(2010)

In [ ]:
df11_n

In [ ]:
df_n3=df11_n.head(1000)

In [ ]:
df_n3.tail(57)

In [ ]:
df11_n[df11_n["Adresse"].str.contains("Löberenstrasse 5, Zug")==True]

In [ ]:
df_fehlende=pd.read_csv("BRFrames/Stadt_Zug_fehlende_adressem")

In [ ]:
df_fehltest=df_fehlende.tail(20)

In [ ]:
listeadress=[]
counter=6000
for adress in tqdm(df_fehlende["Adresse"]): #die Adressen durchgehen
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.send_keys(adress) #Adresse Eingeben
    search_adress.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(9)
    page = driver.page_source.encode("utf-8") #Page in Variable speichern
    
    with open ("HRSeiten/frontpage"+str(counter)+".htm", "wb+") as file:
        file.write(page)
        file.close()
        time.sleep(1)
    pagename = "frontpage"+str(counter)+".htm"        
    counter=counter+1 
        
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.clear()
    time.sleep(1)
   
    
    minidictadress={"Adresse":adress, "Dokument":pagename}
    listeadress.append(minidictadress)

In [ ]:
dfl11=pd.DataFrame(listeadress)

In [ ]:
dfl11

In [ ]:
dfl11.to_csv("BRFrames/adresse_file_Zug3")

In [ ]:
# Nun von Indexnummer 2632 (St. Johannesstrasse 12, frontpage 3675) 
#den Rest der Stadt Zug abfragen
# von 2010 wollte ich 1000 abfragen. 56 blieben übrig.
# Ich muss also nun noch 1066 abfragen 

In [ ]:
dfn4=df11_n.tail(1066)

In [ ]:
dfn4

In [ ]:
listeadress=[]
counter=3676
for adress in tqdm(dfn4["Adresse"]): #die Adressen durchgehen
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.send_keys(adress) #Adresse Eingeben
    search_adress.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(8)
    page = driver.page_source.encode("utf-8") #Page in Variable speichern
    
    with open ("HRSeiten_neu/frontpage"+str(counter)+".htm", "wb+") as file:
        file.write(page)
        file.close()
        time.sleep(1)
    pagename = "frontpage"+str(counter)+".htm"        
    counter=counter+1 
        
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.clear()
    time.sleep(1)
   
    
    minidictadress={"Adresse":adress, "Dokument":pagename}
    listeadress.append(minidictadress)

In [ ]:
dfl114=pd.DataFrame(listeadress)

In [ ]:
dfl114.to_csv("BRFrames/adresse_file_Zug4")

# Versuch das Umlaut-Problem wegzukriegen

In [ ]:
df_fehlende=pd.read_csv("BRFrames/Stadt_Zug_fehlende_adressem")

In [ ]:
df_fehltest=df_fehlende.tail(20)

In [ ]:
listeadress=[]
counter=0
for adress in tqdm(df_fehltest["Adresse"]): #die Adressen durchgehen
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.send_keys(adress) #Adresse Eingeben
    search_adress.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(9)
    page = driver.get.encode("utf-8") #Page in Variable speichern
    
    with open ("HRSeiten_Test/umlauttest"+str(counter)+".htm", "wb+") as file:
        file.write(page)
        file.close()
        time.sleep(1)
    pagename = "frontpage"+str(counter)+".htm"        
    counter=counter+1 
        
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.clear()
    time.sleep(1)
   
    
    minidictadress={"Adresse":adress, "Dokument":pagename}
    listeadress.append(minidictadress)

### Grosser Scraper für ganzen Kanton
Ich plane, die Frames in "fressbare Stückchen" zu zerteilen, und durch die Liste der Dataframes zu loopen. Dabei soll immer wieder eine Pause von Frame zu Frame gemacht werden. Und das Fenster neu geöffnet.

In [ ]:
#Frames für Test
df1=pd.read_csv("Datensätze Adressen/1701.csv",sep=';') #3211 Baar
df2=pd.read_csv("Datensätze Adressen/1702.csv",sep=';') #2127 Cham
df3=pd.read_csv("Datensätze Adressen/1703.csv",sep=';') #1599 Hünenberg

In [ ]:
#Testdf
df1_t=df1.head(10)
df2_t=df2.head(10)
df3_t=df3.head(10)
df1_t["Adresse"]=df1_t["STRNAME"]+" "+df1_t["DEINR"]+", "+ df1_t["PLZNAME"]
df2_t["Adresse"]=df2_t["STRNAME"]+" "+df2_t["DEINR"]+", "+ df2_t["PLZNAME"]
df3_t["Adresse"]=df3_t["STRNAME"]+" "+df3_t["DEINR"]+", "+ df3_t["PLZNAME"]

In [2]:
#Frames kreieren aus Gemeinde-Dataframe für Loop
dff_g=pd.read_csv(folderpath+"/BRFrames/alle_Adressen_ohne_Stadt")

In [3]:
dff_g

,Unnamed: 0,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP,Adresse
0,0,310003,0,ZG,1701,Baar,Marktgasse,12,6340,0,Baar,2682511.588,1227773.739,de,"Marktgasse 12, Baar"
1,1,310004,0,ZG,1701,Baar,Marktgasse,14,6340,0,Baar,2682523.028,1227784.067,de,"Marktgasse 14, Baar"
2,2,310005,0,ZG,1701,Baar,Marktgasse,16,6340,0,Baar,2682540.091,1227792.603,de,"Marktgasse 16, Baar"
3,3,310006,0,ZG,1701,Baar,Marktgasse,18,6340,0,Baar,2682554.253,1227799.894,de,"Marktgasse 18, Baar"
4,4,310007,0,ZG,1701,Baar,Rigistrasse,2,6340,0,Baar,2682578.674,1227789.851,de,"Rigistrasse 2, Baar"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
15657,789,191581692,0,ZG,1710,Walchwil,Hinterbalis,NaN,6318,0,Walchwil,2682918.805,1218133.202,de,"Hinterbalis, Walchwil"
15658,818,191696853,0,ZG,1710,Walchwil,Sattel,NaN,6318,0,Walchwil,2682704.700,1219514.850,de,"Sattel, Walchwil"
15659,850,191730019,0,ZG,1710,Walchwil,Freudenberg,NaN,6318,0,Walchwil,2681806.510,1218036.750,de,"Freudenberg, Walchwil"
15660,873,191730101,0,ZG,1710,Walchwil,Mittelschwändi,NaN,6318,0,Walchwil,2682263.160,1218945.140,de,"Mittelschwändi, Walchwil"


In [4]:
# Nun das Gemeinde-Dataframe miot 16740 Rows in 1000-er-Stücke teilen. (würde sicher auch automatisiert gehen...)
dff1=dff_g.iloc[:1000,:]
dff2=dff_g.iloc[1000:2000,:]
dff3=dff_g.iloc[2000:3000,:]
dff33=dff_g.iloc[2500:3000,:]
dff4=dff_g.iloc[3000:4000,:]
dff5=dff_g.iloc[4000:5000,:]
dff6=dff_g.iloc[5000:6000,:]
dff7=dff_g.iloc[6000:7000,:]
dff8=dff_g.iloc[7000:8000,:]
dff9=dff_g.iloc[8000:9000,:]
dff10=dff_g.iloc[9000:10000,:]
dff11=dff_g.iloc[10000:11000,:]
dff12=dff_g.iloc[11000:12000,:]
dff13=dff_g.iloc[12000:13000,:]
dff14=dff_g.iloc[13000:14000,:]
dff15=dff_g.iloc[14000:15000,:]
dff16=dff_g.iloc[15000:16000,:]
dff17=dff_g.iloc[16000:,:]
#[df1,df2,df3,df4,df5,df6,df7,df8,df9,df10,df11,df12,df13,df14,df15,df16,df17]

In [8]:
dff6

,Unnamed: 0,EGID,EDID,GDEKT,GDENR,GDENAME,STRNAME,DEINR,PLZ4,PLZZ,PLZNAME,GKODE,GKODN,STRSP,Adresse
5000,2091,9033537,1,ZG,1702,Cham,Johannisstrasse,21,6330,0,Cham,2677593.329,1226238.296,de,"Johannisstrasse 21, Cham"
5001,2092,11528377,1,ZG,1702,Cham,Langackerstrasse,6,6330,0,Cham,2676952.478,1227920.314,de,"Langackerstrasse 6, Cham"
5002,2093,11528456,1,ZG,1702,Cham,Rehhalde,19b,6332,0,Hagendorn,2675587.857,1228699.863,de,"Rehhalde 19b, Hagendorn"
5003,2094,190105654,1,ZG,1702,Cham,Dorfplatz,3,6330,0,Cham,2677384.057,1226035.129,de,"Dorfplatz 3, Cham"
5004,2095,190166053,1,ZG,1702,Cham,Zugerstrasse,31b,6330,0,Cham,2677658.627,1226155.838,de,"Zugerstrasse 31b, Cham"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,959,3120365,0,ZG,1703,Hünenberg,Schauburg,1a,6331,0,Hünenberg,2674634.717,1225641.696,de,"Schauburg 1a, Hünenberg"
5996,960,3120366,0,ZG,1703,Hünenberg,Seemattstrasse,3b,6333,0,Hünenberg See,2676719.331,1225258.885,de,"Seemattstrasse 3b, Hünenberg See"
5997,961,3120367,0,ZG,1703,Hünenberg,Seemattstrasse,32,6333,0,Hünenberg See,2676854.166,1225238.898,de,"Seemattstrasse 32, Hünenberg See"
5998,962,3120368,0,ZG,1703,Hünenberg,Seemattstrasse,44,6333,0,Hünenberg See,2676886.449,1225311.575,de,"Seemattstrasse 44, Hünenberg See"


In [3]:
dff1=dff_g.iloc[:1000,:]
dff2=dff_g.iloc[1000:2000,:]
dff3=dff_g.iloc[2000:3000,:]
dff33=dff_g.iloc[2500:3000,:]
dff34=dff_g.iloc[2530:3000,:]
dff4=dff_g.iloc[3080:3200,:]
dff5=dff_g.iloc[3201:3400,:]
dff6=dff_g.iloc[3400:3800,:]
dff7=dff_g.iloc[4160:4200,:]
dff8=dff_g.iloc[4200:4600,:]
dff9=dff_g.iloc[4600:5000,:]
dff10=dff_g.iloc[5000:5400,:]
dff11=dff_g.iloc[5908:6200,:]
dff12=dff_g.iloc[6200:6600,:]
dff13=dff_g.iloc[6600:7000,:]
dff14=dff_g.iloc[7000:14000,:]
dff15=dff_g.iloc[14000:15000,:]
dff16=dff_g.iloc[15000:16000,:]
dff17=dff_g.iloc[16000:,:]

In [5]:
listeadress=[]
counter=4000
nummer=12

df_list=[dff13,dff14,dff15,dff16,dff17]

for df in tqdm(df_list):
    #Wir starten den Browser auf
    driver = webdriver.Chrome(executable_path='C:\chromedriver.exe') #bei Windows ist das anders
    #Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
    driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
    time.sleep(5)
    #Hier drücke ich den Cookie-Banner weg
    cookie=driver.find_element_by_id("j_idt198")
    time.sleep(4)            
    cookie.click()#ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
    time.sleep(2)
    #Ich öffne das zusätzliche Panel, ohne dass man dies auf der Seite sieht
    #su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
    #time.sleep(2)
    #driver.execute_script('arguments[0].click();', su_fel_ad)
    #time.sleep(2)
    # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
    #filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
    #filt.send_keys('Domiziladresse')
    #filt.send_keys(Keys.ENTER)
    #time.sleep(2)
    # Hier klappe ich die Zusätzlichen Suchfelder auf
    driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
    time.sleep(3)
    for adress in tqdm(df["Adresse"]): #die Adressen durchgehen
        # Den Filter ansteuern, wo ich Domiziladresse auwählen kann
        su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
        time.sleep(2)
        driver.execute_script('arguments[0].click();', su_fel_ad)
        time.sleep(2)
        filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
        time.sleep(2)
        try:
            filt.send_keys('Domiziladresse')
        except StaleElementReferenceException:
            print("Domizilerror")
            filt.send_keys('Domiziladresse')
       
            filt.send_keys(Keys.ENTER)
            time.sleep(2)
    
        search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
        try:# Ich mache eine Try-Schlaufe, da er hier mehrmals einen Error schmiss
            search_adress.send_keys(adress) #Adresse Eingeben
           
        except ElementNotInteractableException: #Ich gebe ihm ein zweites Mal Zeit
            print("nicht interactable")
            search_adress.send_keys(adress) #Adresse Eingeben
        except ElementNotInteractableException: 
            print(adress+"nicht interactable break")
            break
        
        search_adress.send_keys(Keys.ENTER) #Enter drücken
        time.sleep(5)
        
        page = driver.page_source.encode("utf-8") #Page in Variable speichern

        with open (folderpath+"/HRSeiten_Gemeinden/frontpage"+str(nummer)+"_"+str(counter)+".htm", "wb+") as file:
            file.write(page)
            file.close()
            
        pagename = "frontpage"+str(nummer)+"_"+str(counter)+".htm"        
        counter=counter+1 
        try: #Hier eine Try-Schlaufe weil auch das Clear nicht immer funktioniert. hat sich bewährt
            clear_all=driver.find_element_by_id("idSucheForm:j_idt167").click()
            time.sleep(3)
        except StaleElementReferenceException:
            clear_all=driver.find_element_by_id("idSucheForm:j_idt167").click()
            time.sleep(3)
            print("geht nicht")
        except exceptions.StaleElementReferenceException:
            print("voll durch")
            break

        
        
        nummer=nummer+1
    driver.close() # Hier das Browserfenster wieder schliessen, um danach für das neue df erneut zu öffnen
    time.sleep(300) # Nun Pause.

        



  0%|          | 0/5 [00:00<?, ?it/s]


  0%|          | 0/400 [00:00<?, ?it/s]


  0%|          | 1/400 [00:14<1:37:22, 14.64s/it]


  0%|          | 2/400 [00:29<1:36:58, 14.62s/it]


  1%|          | 3/400 [00:43<1:36:37, 14.60s/it]


  1%|          | 4/400 [00:58<1:36:13, 14.58s/it]


  1%|▏         | 5/400 [01:12<1:36:00, 14.58s/it]


  2%|▏         | 6/400 [01:27<1:35:48, 14.59s/it]


  2%|▏         | 7/400 [01:42<1:35:33, 14.59s/it]


  2%|▏         | 8/400 [01:56<1:35:23, 14.60s/it]


  2%|▏         | 9/400 [02:11<1:35:10, 14.61s/it]


  2%|▎         | 10/400 [02:25<1:34:51, 14.59s/it]


  3%|▎         | 11/400 [02:40<1:34:34, 14.59s/it]


  3%|▎         | 12/400 [02:54<1:34:08, 14.56s/it]


  3%|▎         | 13/400 [03:09<1:33:52, 14.56s/it]


  4%|▎         | 14/400 [03:23<1:33:29, 14.53s/it]


  4%|▍         | 15/400 [03:38<1:33:11, 14.52s/it]


  4%|▍         | 16/400 [03:53<1:33:01, 14.54s/it]


  4%|▍         | 17/400 [04:07<1:32:50, 14.54s/it]


  4%|▍         | 18/400 

 77%|███████▋  | 308/400 [1:14:35<22:17, 14.54s/it]


 77%|███████▋  | 309/400 [1:14:49<22:02, 14.54s/it]


 78%|███████▊  | 310/400 [1:15:04<21:47, 14.53s/it]


 78%|███████▊  | 311/400 [1:15:18<21:32, 14.53s/it]


 78%|███████▊  | 312/400 [1:15:33<21:17, 14.52s/it]


 78%|███████▊  | 313/400 [1:15:47<21:03, 14.52s/it]


 78%|███████▊  | 314/400 [1:16:02<20:49, 14.53s/it]


 79%|███████▉  | 315/400 [1:16:16<20:36, 14.54s/it]


 79%|███████▉  | 316/400 [1:16:31<20:22, 14.55s/it]


 79%|███████▉  | 317/400 [1:16:45<20:08, 14.56s/it]


 80%|███████▉  | 318/400 [1:17:00<19:54, 14.56s/it]


 80%|███████▉  | 319/400 [1:17:15<19:39, 14.57s/it]


 80%|████████  | 320/400 [1:17:29<19:24, 14.56s/it]


 80%|████████  | 321/400 [1:17:44<19:09, 14.54s/it]


 80%|████████  | 322/400 [1:17:58<18:54, 14.55s/it]


 81%|████████  | 323/400 [1:18:13<18:39, 14.54s/it]


 81%|████████  | 324/400 [1:18:27<18:25, 14.54s/it]


 81%|████████▏ | 325/400 [1:18:42<18:10, 14.54s/it]


 82%|████████▏ | 326/400 [1:

  3%|▎         | 204/7000 [49:22<27:25:50, 14.53s/it]


  3%|▎         | 205/7000 [49:36<27:25:35, 14.53s/it]


  3%|▎         | 206/7000 [49:51<27:27:06, 14.55s/it]


  3%|▎         | 207/7000 [50:05<27:26:36, 14.54s/it]


  3%|▎         | 208/7000 [50:20<27:25:12, 14.53s/it]


  3%|▎         | 209/7000 [50:34<27:26:02, 14.54s/it]


  3%|▎         | 210/7000 [50:49<27:25:20, 14.54s/it]


  3%|▎         | 211/7000 [51:03<27:27:05, 14.56s/it]


  3%|▎         | 212/7000 [51:18<27:24:53, 14.54s/it]


  3%|▎         | 213/7000 [51:32<27:23:47, 14.53s/it]


  3%|▎         | 214/7000 [51:47<27:23:37, 14.53s/it]


  3%|▎         | 215/7000 [52:01<27:21:35, 14.52s/it]


  3%|▎         | 216/7000 [52:16<27:22:02, 14.52s/it]


  3%|▎         | 217/7000 [52:30<27:20:16, 14.51s/it]


  3%|▎         | 218/7000 [52:45<27:22:58, 14.54s/it]


  3%|▎         | 219/7000 [53:00<27:24:15, 14.55s/it]


  3%|▎         | 220/7000 [53:14<27:22:55, 14.54s/it]


  3%|▎         | 221/7000 [53:29<27:23:35, 14.55

NameError: name 'ElementNotInteractableException' is not defined

In [ ]:
listeadress

In [ ]:
#Wir starten den Browser auf
driver = webdriver.Chrome(executable_path='C:\chromedriver.exe') #bei Windows ist das anders
#Und nun sagen wir dem Browser, welche Seite er besuchen sollte. 
driver.get('https://zg.chregister.ch/cr-portal/suche/suche.xhtml')
time.sleep(3)
#Hier drücke ich den Cookie-Banner weg
driver.find_element_by_id("j_idt192").click() #ACHTUNG: Möglicherweise wechselt hier immer mal wieder die ID-NR
#Ich öffne das zusätzliche Panel, ohne dass man dies auf der Seite sieht
su_fel_ad = driver.find_element_by_id('idSucheForm:panel:idDiverseTyp_label')
driver.execute_script('arguments[0].click();', su_fel_ad)
time.sleep(2)
# Den Filter ansteuern, wo ich Domiziladresse auwählen kann
filt = driver.find_element_by_id("idSucheForm:panel:idDiverseTyp_filter")
filt.send_keys('Domiziladresse')
filt.send_keys(Keys.ENTER)
time.sleep(2)
# Hier klappe ich die Zusätzlichen Suchfelder auf
driver.find_element_by_class_name('ui-icon.ui-icon-triangle-1-e').click()
time.sleep(2)

In [ ]:
listeadress=[]
counter=503
number=3
for adress in tqdm(dff33["Adresse"]): #die Adressen durchgehen
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.send_keys(adress) #Adresse Eingeben
    search_adress.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(9)
    page = driver.page_source.encode("utf-8") #Page in Variable speichern
    
    with open ("HRSeiten_Gemeinden/frontpage"+str(number)+str(counter)+".htm", "wb+") as file:
        file.write(page)
        file.close()
        time.sleep(1)
    pagename = "HRSeiten_Gemeinden/frontpage"+str(number)+str(counter)+".htm"        
    counter=counter+1 
        
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.clear()
    time.sleep(1)
   
    
    minidictadress={"Adresse":adress, "Dokument":pagename}
    listeadress.append(minidictadress)

# MISSLUNGENER VERSUCH ALLES IN EINEM LOOP

In [ ]:
http://localhost:8888/tree/Desktop/maz-datenjournalismus/Daten_BR_Handelsreg

In [ ]:
# Nun alles im Loop
listeadress=[]
counter=0
for adress in dff_test["Adresse"]: #die Adressen durchgehen
    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.send_keys(adress) #Adresse Eingeben
    search_adress.send_keys(Keys.ENTER) #Enter drücken
    time.sleep(5)
    try:
        page = driver.page_source.encode("utf-8") #Page in Variable speichern
        driver.find_element_by_class_name("ui-paginator-next.ui-state-default.ui-corner-all").click()
        pagetest= driver.page_source.encode("utf-8") #Page next in Variable abspeichern
        
        if page == pagetest == False:
            pagename= "page"+str(counter) #Speichername erstellen
            #Hier zurück auf die gewsünschte SEite:
            driver.find_element_by_class_name("ui-paginator-prev.ui-state-default.ui-corner-all.ui-state-disabled") 
            page_fix = driver.page_source.encode("utf-8") #Page nun abspeichern
            time.sleep(1)
            with open ("HRSeiten/"+pagename+".htm", "wb+") as file:
                file.write(page_fix)
                file.close()
            time.sleep(1)
            driver.find_element_by_class_name("ui-paginator-next.ui-state-default.ui-corner-all").click() #eine SEite weiter
            counter=counter+1 
        else:
            driver.find_element_by_class_name("ui-paginator-first.ui-state-default.ui-corner-all").click() #auf die erste Eite
            pass

    except: 
        pagename="Keine Firma"
        pass

    search_adress=driver.find_element_by_id("idSucheForm:panel:idDiverseSuchtext")
    search_adress.clear()
    
    minidictadress={"Adresse":adress, "Dokument":pagename}
    listeadress.append(minidictadress)